<img src="https://upload.wikimedia.org/wikipedia/fr/8/81/Sciences_SU.png" width="240" height="240" align="center"/>


# MLA 701. Du RNN au deep RNN [prédiction de séries temporelles]

In [ ]:
# Dans ce notebook, nous allons nous intéresser à des tâches d'analyse prédictive de séries temporelles
# Par exemple, pour prédire la météo de demain ou le cours de la bourse

# Ce notebook vise à apprendre : 
# - L'organisation des données de séries temporelles numériques en batches
# - La compréhension des architectures RNN et la capacité à réaliser le schéma d'un réseau RNN et d'en expliciter
# les paramètres et les méta-paramètres
# - La comparaison d'un RNN à la vanille avec un RNN profond


In [ ]:
# On importe les librairies de base pour le calcul et l'affichage
import numpy as np
import math
import matplotlib.pyplot as plt

n_time  = 50      # longueur de la série temporelle
n_batch = 1000    # nombre de batches
# proportion des données dans les ensembles d'entrainement, validation, et test
r_trn   = 0.8     
r_val   = 0.1 
r_tst   = 0.1
# taille des ensembles
n_trn = int(r_trn * n_batch)
n_val = int(r_val * n_batch)
n_tst = int(r_tst * n_batch)

# Fonction pour créer nbatch séries temporelles de longueur n_time
def create_time_serie(batch_size, n_time):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_time)
    serie = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))    # + sinus 1
    serie += 0.4 * np.sin((time - offsets2) * (freq2 * 20 + 20))   # + sinus 2
    serie += 0.1 * (np.random.rand(batch_size, n_time) - 0.5)      # + bruit aléatoire
    return serie[..., np.newaxis].astype(np.float32)

# On créée un jeu de séries temporelles
serie = create_time_serie(n_batch, n_time + 1)

# On crée les ensembles
X_train, y_train = serie[:n_trn, :n_time], serie[:n_trn, -1]
X_valid, y_valid = serie[n_trn:n_trn+n_val, :n_time], serie[n_trn:n_trn+n_val, -1]
X_test, y_test   = serie[n_trn+n_val:, :n_time], serie[n_trn+n_val:, -1]

# Question : à partir de l'organisation des données d'entrée/sortie, 
# quelle est la tâche que l'on cherche à résoudre? 

In [ ]:
# Pour répondre à cette question, on trace un batch de données 
i_batch = 100
plt.plot(X_train[i_batch])
plt.plot(X_train[i_batch], 'bo', fillstyle='none', markersize=8)
plt.plot(n_time, y_train[i_batch], 'r+',  markersize=8)
plt.xlabel("time [in samples]")

# Donc?

In [ ]:
# On importe les librairies de TF pour l'apprentissage
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

# On créé quelques paramètres d'entrée du réseau
hid_dim    = 1
n_epoch    = 20

# On instancie le réseau
RNN_model  = Sequential(name="RNN_model")
RNN_model.add(SimpleRNN(hid_dim, input_shape=[None, 1], stateful=False, return_sequences=False, name='RNN'))

# Questions : 
# 1) Réaliser le schéma de ce réseau
# Et Expliciter les méta-paramètres de ce réseau (y compris ceux utilisés par défaut)
# 2) Que signifient les parmaètres : "stateful=False" et "return_sequences=False" ?
# 3) Est-ce justifié dans ce cas? Expliquer.

# On compile le réseau
RNN_model.compile(loss='mse', optimizer='adam')

# On visualise la structure du réseau
RNN_model.summary()

# On apprend le modèle
history = RNN_model.fit(X_train, y_train, epochs=n_epoch,
                    validation_data=(X_valid, y_valid))

L_val = RNN_model.evaluate(X_valid, y_valid)

print("")
print("La loss MSE sur le jeu de validation est de: {}" .format(L_val))


In [ ]:
# On prédit sur l'ensemble de test
y_hat = RNN_model.predict(X_test)

# On trace un résultat
i_batch = 0
plt.plot(X_test[i_batch])
plt.plot(X_test[i_batch], 'bo', fillstyle='none', markersize=8)
plt.plot(n_time, y_test[i_batch], 'g+',  markersize=8)
plt.plot(n_time, y_hat[i_batch], 'ro',  fillstyle='none', markersize=8)
plt.xlabel("time [in samples]")

# On évalue la loss sur l'ensemble de test
L_tst = RNN_model.evaluate(X_test, y_test)

print("")
print("La loss MSE sur l'ensemble de test est de: {}" .format(L_tst))

# Question : 
# 1) la prédiction (alt. la performance) est-elle satisfaisante? Justifier.
# 2) proposer une solution pour améliorer la performance du réseau


In [ ]:
# On recommence mais avec un réseau profond (3 couches RNN) et plus de neurones par couche
# On créé quelques paramètres d'entrée du réseau
hid_dim    = 10
n_epoch    = 20

# On instancie le réseau
DRNN_model  = Sequential(name="DRNN_model")
DRNN_model.add(SimpleRNN(hid_dim, input_shape=[None, 1], return_sequences=True, name='RNN_layer1'))
DRNN_model.add(SimpleRNN(hid_dim, return_sequences=True, name='RNN_layer2'))
DRNN_model.add(SimpleRNN(1, return_sequences=False, name='RNN_layer3'))

# Question : 
# Même questions que précédemment mais pour les 3 couches

# On compile le réseau
DRNN_model.compile(loss='mse', optimizer='adam')

# On visualise la structure du réseau
DRNN_model.summary()

# On apprend le modèle
history = DRNN_model.fit(X_train, y_train, epochs=n_epoch,
                    validation_data=(X_valid, y_valid))

L_val = DRNN_model.evaluate(X_valid, y_valid)

print("")
print("La loss MSE sur le jeu de validation est de: {}" .format(L_val))

In [ ]:
# On prédit sur l'ensemble de test
y_hat = DRNN_model.predict(X_test)

# On trace un résultat
i_batch = 0
plt.plot(X_test[i_batch])
plt.plot(X_test[i_batch], 'bo', fillstyle='none', markersize=8)
plt.plot(n_time, y_test[i_batch], 'g+',  markersize=8)
plt.plot(n_time, y_hat[i_batch], 'ro',  fillstyle='none', markersize=8)
plt.xlabel("time [in samples]")

# On évalue la loss sur l'ensemble de test
L_tst = DRNN_model.evaluate(X_test, y_test)

print("")
print("La loss MSE sur l'ensemble de test est de: {}" .format(L_tst))

# Question : 
# 1) maintenant, la prédiction (alt. la performance) est-elle satisfaisante? Justifier.

In [ ]:
# Question : conclusions?